# Exercises

In this exercise we will look at how substitution affects the vibrational frequencies, IR, and Raman spectra.

In [1]:
# import section
import veloxchem as vlx
import py3Dmol as p3d
from veloxchem.veloxchemlib import bohr_in_angstroms
from matplotlib import pyplot as plt
import numpy as np
from veloxchem.veloxchemlib import bohr_in_angstroms

basis_set_label = "6-31G"

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.
* Warning * Setting MKL_THREADING_LAYER to "GNU".


In [2]:
# define molecules and basis sets
ethene_xyz = """6
 ethene
 C          0.000000    -0.663984   0.000000
 C          0.000000     0.663984   0.000000
 H          0.919796    -1.223061   0.000000
 H         -0.919796    -1.223061   0.000000
 H          0.919796     1.223061   0.000000
 H         -0.919796     1.223061   0.000000
"""
ethene = vlx.Molecule.from_xyz_string(ethene_xyz)
ethene_basis = vlx.MolecularBasis.read(ethene, basis_set_label)

In [3]:
fluoroethene_xyz = """6
fluoroethene
 C          0.000000    -0.663984   0.000000
 C          0.000000     0.663984   0.000000
 F         1.519796    -1.223061   0.000000
 H         -0.919796    -1.223061   0.000000
 H          0.919796     1.223061   0.000000
 H         -0.919796     1.223061   0.000000
 """
fluoroethene = vlx.Molecule.from_xyz_string(fluoroethene_xyz)
fluoroethene_basis = vlx.MolecularBasis.read(fluoroethene, basis_set_label)

In [4]:
chloroethene_xyz = """6
chloroethene
 C          0.000000    -0.663984   0.000000
 C          0.000000     0.663984   0.000000
 Cl         1.519796    -1.223061   0.000000
 H         -0.919796    -1.223061   0.000000
 H          0.919796     1.223061   0.000000
 H         -0.919796     1.223061   0.000000
 """
chloroethene = vlx.Molecule.from_xyz_string(chloroethene_xyz)
chloroethene_basis = vlx.MolecularBasis.read(chloroethene, basis_set_label)

In [5]:
# to view the molecules
view = p3d.view(linked=True, viewergrid=(1,3),width=700,height=200)
view.addModel(ethene_xyz, 'xyz', viewer=(0,0))
view.addModel(fluoroethene_xyz, 'xyz', viewer=(0,1))
view.addModel(chloroethene_xyz, 'xyz', viewer=(0,2))
view.setStyle({'stick': {}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## SCF geometry optimization
Before we can calculate the vibrational spectra, we first must optimize the geometries.

In [6]:
# Settings for SCF and gradient drivers
scf_settings = {'conv_thresh':1e-6}
method_settings = {}

In [7]:
# Run SCF for ethene
ethene_scfdrv = vlx.ScfRestrictedDriver()
ethene_scfdrv.update_settings(scf_settings, method_settings)
ethene_scfdrv.ostream.state = False 
ethene_scfdrv.compute(ethene, ethene_basis)

# Run SCF for fluoroethene
fluoroethene_scfdrv = vlx.ScfRestrictedDriver()
fluoroethene_scfdrv.update_settings(scf_settings, method_settings)
fluoroethene_scfdrv.ostream.state = False 
fluoroethene_scfdrv.compute(fluoroethene, fluoroethene_basis)

# Run SCF for chloroethene
chloroethene_scfdrv = vlx.ScfRestrictedDriver()
chloroethene_scfdrv.update_settings(scf_settings, method_settings)
chloroethene_scfdrv.ostream.state = False 
chloroethene_scfdrv.compute(chloroethene, chloroethene_basis)

In [8]:
# Set up the gradient and optimization dirvers:
ethene_grad_drv = vlx.ScfGradientDriver(ethene_scfdrv)
ethene_opt_drv = vlx.OptimizationDriver(ethene_grad_drv)

fluoroethene_grad_drv = vlx.ScfGradientDriver(fluoroethene_scfdrv)
fluoroethene_opt_drv = vlx.OptimizationDriver(fluoroethene_grad_drv)

chloroethene_grad_drv = vlx.ScfGradientDriver(chloroethene_scfdrv)
chloroethene_opt_drv = vlx.OptimizationDriver(chloroethene_grad_drv)

In [9]:
# Either optimize the geometries
#opt_ethene = ethene_opt_drv.compute(ethene, ethene_basis)

KeyboardInterrupt: 

In [ ]:
fluoroethene_opt_drv.ostream.state = True # to enable printout
#opt_fluoroethene = fluoroethene_opt_drv.compute(fluoroethene, fluoroethene_basis)

In [ ]:
chloroethene_opt_drv.ostream.state = True
#opt_chloroethene = chloroethene_opt_drv.compute(chloroethene, chloroethene_basis)

In [ ]:
# Get optimized coordinates
def get_xyz(molecule):
    natm = molecule.number_of_atoms()
    elements = molecule.get_labels()
    coords = molecule.get_coordinates() * bohr_in_angstroms()
    txt = "%d\n\n" % natm
    for i in range(natm):
        txt += elements[i] + " %15.7f %15.7f %15.7f\n" % (coords[i,0], coords[i,1], coords[i,2])
    return txt

In [ ]:
opt_ethene_xyz = get_xyz(opt_ethene)
opt_fluoroethene_xyz = get_xyz(opt_fluoroethene)
opt_chloroethene_xyz = get_xyz(opt_chloroethene)

```python
# Or read optimized geometries from file:
opt_ethene_xyz = open("../xyz/ir_raman/opt_ethene.xyz", "r").read() # read optimized geometry from file
opt_fluoroethene_xyz = open("../xyz/ir_raman/opt_fluoroethene.xyz", "r").read() # read optimized geometry from file
opt_chloroethene_xyz = open("../xyz/ir_raman/opt_chloroethene.xyz", "r").read() # read optimized geometry from file

opt_ethene = vlx.Molecule.from_xyz_string(opt_ethene_xyz)
ethene_basis = vlx.MolecularBasis.read(opt_ethene, basis_set_label)

opt_fluoroethene = vlx.Molecule.from_xyz_string(opt_fluoroethene_xyz)
fluoroethene_basis = vlx.MolecularBasis.read(opt_fluoroethene, basis_set_label)

opt_chloroethene = vlx.Molecule.from_xyz_string(opt_chloroethene_xyz)
chloroethene_basis = vlx.MolecularBasis.read(opt_chloroethene, basis_set_label)
```

In [ ]:
# Visualize the optimized molecular structures
view = p3d.view(linked=True, viewergrid=(1,3),width=700,height=200)
view.addModel(opt_ethene_xyz, 'xyz', viewer=(0,0))
view.addModel(opt_fluoroethene_xyz, 'xyz', viewer=(0,1))
view.addModel(opt_chloroethene_xyz, 'xyz', viewer=(0,2))
view.setStyle({'stick': {}})
view.zoomTo()
view.show()

In [ ]:
# Compute SCF with the optimized geometries of all molecules
ethene_scfdrv.compute(opt_ethene, ethene_basis)
fluoroethene_scfdrv.compute(opt_fluoroethene, fluoroethene_basis)
chloroethene_scfdrv.compute(opt_chloroethene, chloroethene_basis)

## Hessians

Now, let's calculate the IR and Raman spectra for these optimized geometries

In [ ]:
# Settings for Hessian calculation
hessian_settings = {'do_raman': 'yes', 'print_depolarization_ratio':'no'}

In [ ]:
# Create Hessian driver and update settings
ethene_hessian_drv = vlx.scfhessiandriver.ScfHessianDriver(ethene_scfdrv)
ethene_hessian_drv.update_settings(method_settings, hessian_settings)

fluoroethene_hessian_drv = vlx.scfhessiandriver.ScfHessianDriver(fluoroethene_scfdrv)
fluoroethene_hessian_drv.update_settings(method_settings, hessian_settings)

chloroethene_hessian_drv = vlx.scfhessiandriver.ScfHessianDriver(chloroethene_scfdrv)
chloroethene_hessian_drv.update_settings(method_settings, hessian_settings)

In [ ]:
# Compute the Hessians:
ethene_hessian_drv.compute(ethene, ethene_basis)
fluoroethene_hessian_drv.compute(fluoroethene, fluoroethene_basis)
chloroethene_hessian_drv.compute(chloroethene, chloroethene_basis)

```python
# Or read them from file:
import h5py

fname = '../h5files/ir_raman/ethenes.h5'
hf = h5py.File(fname, "r")

labels = ['ethene', 'fluoroethene', 'chloroethene']

i = 0   
for driver in [ethene_hessian_drv, fluoroethene_hessian_drv, chloroethene_hessian_drv]:
    label = labels[i]
    
    driver.hessian = np.array(hf.get(label + '_hessian')) 
    driver.dipole_gradient = np.array( hf.get(label + '_dipolegrad'))
    driver.polarizability_gradient = np.array(hf.get(label + '_polgrad')) 
    
    i += 1

hf.close()
```

In [ ]:
# Broadening function
def add_broadening(list_ex_energy, list_osci_strength, line_profile='Lorentzian', line_param=10, step=10):
        x_min = np.amin(list_ex_energy) - 50
        x_max = np.amax(list_ex_energy) + 50
        x = np.arange(x_min, x_max, step)
        y = np.zeros((len(x)))
        #print(x)
        #print(y)

        # go through the frames and calculate the spectrum for each frame
        for xp in range(len(x)):
            for e, f in zip(list_ex_energy, list_osci_strength):
                if line_profile == 'Gaussian':
                    y[xp] += f * np.exp(-(
                        (e - x[xp]) / line_param)**2)
                elif line_profile == 'Lorentzian':
                    y[xp] += 0.5 * line_param * f / (np.pi * (
                        (x[xp] - e)**2 + 0.25 * line_param**2))
        return x, y

        
# To animate the normal mode we will need both the geometry and the displacements 
def get_normal_mode(molecule, normal_mode):
    elements = molecule.get_labels()
    coords = molecule.get_coordinates() * bohr_in_angstroms() # To transform from au to A
    natm = molecule.number_of_atoms()
    vib_xyz = "%d\n\n" % natm
    nm = normal_mode.reshape(natm, 3)
    for i in range(natm):
        # add coordinates:
        vib_xyz += elements[i] + " %15.7f %15.7f %15.7f " % (coords[i,0], coords[i,1], coords[i,2])
        # add displacements:
        vib_xyz += "%15.7f %15.7f %15.7f\n" % (nm[i,0], nm[i,1], nm[i,2])
    return vib_xyz

## Vibrational analysis

To get a summary of the vibrational analysis, one simply needs to run the following command:

In [ ]:
# Run for all molecules
ethene_hessian_drv.vibrational_analysis(opt_ethene)
fluoroethene_hessian_drv.vibrational_analysis(opt_fluoroethene)
chloroethene_hessian_drv.vibrational_analysis(opt_chloroethene)

In [ ]:
# Plot the IR spectra
plt.figure(figsize=(7,4))

eth_x, eth_ir = ethene_hessian_drv.frequencies, ethene_hessian_drv.ir_intensities
flo_x, flo_ir = fluoroethene_hessian_drv.frequencies, fluoroethene_hessian_drv.ir_intensities
chl_x, chl_ir = chloroethene_hessian_drv.frequencies, chloroethene_hessian_drv.ir_intensities

eth_xl, eth_irl = add_broadening(eth_x, eth_ir, line_profile='Gaussian', line_param=20, step=2)
flo_xl, flo_irl = add_broadening(flo_x, flo_ir, line_profile='Gaussian', line_param=20, step=2)
chl_xl, chl_irl = add_broadening(chl_x, chl_ir, line_profile='Gaussian', line_param=20, step=2)

plt.plot(eth_xl, eth_irl, label='Ethene')
plt.plot(eth_x, eth_ir, 'x', label='Ethene')

plt.plot(flo_xl, flo_irl, label='Fluoroethene')
plt.plot(flo_x, flo_ir, '*', label='Fluoroethene')

plt.plot(chl_xl, chl_irl, label='Chloroethene')
plt.plot(chl_x, chl_ir, 'o' , label='Chloroethene')

plt.xlabel('Wavenumber (cm**-1)')
plt.axis(xmin=3200, xmax=3500)
#plt.axis(ymin=-0.2, ymax=1.5)
plt.axis(ymin=-0.2, ymax=50)
plt.ylabel('IR intensity (km/mol)')
plt.title("Calculated IR sepctra of substituted ethenes")
plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
# Plot the IR spectra
plt.figure(figsize=(7,4))

eth_x, eth_ir = ethene_hessian_drv.frequencies, ethene_hessian_drv.raman_intensities
flo_x, flo_ir = fluoroethene_hessian_drv.frequencies, fluoroethene_hessian_drv.raman_intensities
chl_x, chl_ir = chloroethene_hessian_drv.frequencies, chloroethene_hessian_drv.raman_intensities

eth_xl, eth_irl = add_broadening(eth_x, eth_ir, line_profile='Gaussian', line_param=20, step=2)
flo_xl, flo_irl = add_broadening(flo_x, flo_ir, line_profile='Gaussian', line_param=20, step=2)
chl_xl, chl_irl = add_broadening(chl_x, chl_ir, line_profile='Gaussian', line_param=20, step=2)

plt.plot(eth_xl, eth_irl, label='Ethene')
plt.plot(eth_x, eth_ir, 'x', label='Ethene')

plt.plot(flo_xl, flo_irl, label='Fluoroethene')
plt.plot(flo_x, flo_ir, '*', label='Fluoroethene')

plt.plot(chl_xl, chl_irl, label='Chloroethene')
plt.plot(chl_x, chl_ir, 'o' , label='Chloroethene')

plt.xlabel('Wavenumber (cm**-1)')
plt.axis(xmin=3200, xmax=3500)
#plt.axis(ymin=-0.2, ymax=1.5)
#plt.axis(ymin=-0.2, ymax=50)
plt.ylabel('Raman activities')
plt.title("Calculated Raman sepctra of substituted ethenes")
plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
# Get the displacements of the normal mode
ethene_h1 = get_normal_mode(ethene, ethene_hessian_drv.normal_modes[-1])
fluoroethene_h1 = get_normal_mode(fluoroethene, fluoroethene_hessian_drv.normal_modes[-3])
chloroethene_h1 = get_normal_mode(chloroethene, chloroethene_hessian_drv.normal_modes[-3])

In [ ]:
# # Animate the vibration
# view = p3d.view(viewergrid=(1,1), width=300, height=200, linked=True)
# view.addModel(fluoroethene_h1, "xyz", {'vibrate': {'frames':10,'amplitude':0.75}}, viewer=(0,0))
# view.setStyle({'stick':{}})
# view.animate({'loop': 'backAndForth'})
# view.zoomTo()
# view.show()

## IR intensities and Raman activities

To rationalize why the IR and Raman spectra look as they do, calculate how the dipole moment and polarizability change during particular vibrational motions. Look at the Hydrogen stretching modes and select a mode which is IR-active in ethene, but is "suppressed" in fluoroethene or chloroethene. What is the dipole moment in the optimized molecule? How does the dipole moment change during the vibration? 

In [ ]:
ethene_prop = vlx.firstorderprop.FirstOrderProperties()
ethene_prop.compute_scf_prop(opt_ethene, ethene_basis, ethene_scfdrv.scf_tensors)
ethene_dipole_moment = ethene_prop.get_property('dipole moment')

ethene_moments = []

In [ ]:
natm = opt_ethene.number_of_atoms()    
new_ethene_scfdrv = vlx.scfrestdriver.ScfRestrictedDriver()
displacements = ethene_hessian_drv.normal_modes[-1].reshape(natm, 3)
labels = opt_ethene.get_labels()

print(displacements.shape)

In [ ]:
for x in [-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]:
    coords = opt_ethene.get_coordinates() * bohr_in_angstroms()
    coords += x * displacements
    new_ethene = vlx.molecule.Molecule(labels, coords, units='angstrom')
    new_ethene_scfdrv.compute(new_ethene, ethene_basis)
    ethene_prop.compute_scf_prop(new_ethene, ethene_basis, new_ethene_scfdrv.scf_tensors)
    ethene_dipole_moment = ethene_prop.get_property('dipole moment')
    ethene_moments.append(ethene_dipole_moment)

In [ ]:
ethene_moments_norms = []

for m in ethene_moments:
    norm = np.linalg.norm(m)
    ethene_moments_norms.append(norm)

In [ ]:
disp = [-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]


plt.plot(disp, ethene_moments_norms, label='Ethene')

plt.xlabel('Displacement (%)')
plt.ylabel('Norm of dipole moment (a.u.)')
plt.title("Dipole Moment along the 3407.94 cm-1 mode")
plt.legend()
plt.tight_layout(); plt.show()

In [ ]:
ethene_lrdrv = vlx.lrsolver.LinearResponseSolver()
ethene_pol_dict = ethene_lrdrv.compute(opt_ethene, ethene_basis, ethene_scfdrv.scf_tensors)

In [ ]:
ethene_pol_dict['response_functions']